# Task 30.04.20 Give Me Some Credit

In [54]:
#Странности (взгляд из Excel)
#NumberOfTime30-59DaysPastDueNotWorse, NumberOfTimes90DaysLate, NumberOfTime60-89DaysPastDueNotWorse = 96 или 98
#Очень большой DebtRatio, как правило, при MonthlyIncome = NA или 0. Импутация MonthlyIncome ситуацию не исправит, т.к. не приведёт к пересчету DebtRatio
#И пр.


In [55]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import GradientBoostingRegressor
#from catboost import CatBoostClassifier
#from sklearn.metrics import roc_auc_score
#from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#from sklearn.model_selection import cross_val_score, cross_val_predict
#from category_encoders import WOEEncoder, SumEncoder

In [56]:
os.getcwd()

'D:\\ML\\Gewissta\\My course\\GiveMeSomeCredit'

In [57]:
os.chdir('D:\\ML\\Gewissta\\My course\\GiveMeSomeCredit')

In [58]:
data = pd.read_csv('cs-training.csv', sep = ',')

In [59]:
data.tail()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
149995,149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0
149999,150000,0,0.850283,64,0,0.249908,8158.0,8,0,2,0,0.0


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
Unnamed: 0                              150000 non-null int64
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(8)
memory usage: 13.7 MB


In [61]:
data.isnull().sum()

Unnamed: 0                                  0
SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [62]:
data.duplicated().sum()

0

In [63]:
num_columns_1 = ['MonthlyIncome']
num_columns_2 = ['NumberOfDependents']
num_columns_3 = ['RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 
                'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 
                'NumberOfTime60-89DaysPastDueNotWorse']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('SeriousDlqin2yrs', axis = 1), data['SeriousDlqin2yrs'], 
                                                   test_size = 0.3, stratify = data['SeriousDlqin2yrs'],
                                                   random_state = 42)

In [65]:
num_pipe_1 = Pipeline([('imp', SimpleImputer(strategy = 'mean')), 
                    ('scaler', StandardScaler())])

In [66]:
num_pipe_2 = Pipeline([('imp', SimpleImputer(strategy = 'most_frequent')), 
                    ('scaler', StandardScaler())])

In [67]:
num_pipe_3 = Pipeline([('scaler', StandardScaler())])

In [68]:
transformers = [('num_1', num_pipe_1, num_columns_1), ('num_2', num_pipe_2, num_columns_2), 
                ('num_3', num_pipe_3, num_columns_3)]

In [69]:
transformer = ColumnTransformer(transformers = transformers)

In [70]:
ml_pipe = Pipeline([('tf', transformer),
                    ('logreg', LogisticRegression(solver = 'lbfgs', max_iter = 200))])

In [71]:
param_grid = {'logreg__C': [.01, .1, .5, 1, 5, 10, 100]}

In [72]:
gs = GridSearchCV(ml_pipe, param_grid, cv = 5, return_train_score = False)

In [73]:
gs.fit(X_train, y_train)
print('best parameters', gs.best_params_)
print('best score', gs.best_score_)
print('Accuracy test', gs.score(X_test, y_test))

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\base.py:465: DataConversionWarning

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarn

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarn

best parameters {'logreg__C': 10}
best score 0.9339619047619048
Accuracy test 0.9338222222222222


C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [74]:
results = pd.DataFrame(gs.cv_results_)
table = results.pivot_table(values=['mean_test_score'],    
                            index=['param_logreg__C'])
print(table)

                 mean_test_score
param_logreg__C                 
0.01                    0.933305
0.10                    0.933819
0.50                    0.933952
1.00                    0.933924
5.00                    0.933952
10.00                   0.933962
100.00                  0.933962
